In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import surprise
from surprise.prediction_algorithms.knns import KNNBaseline
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise.model_selection import cross_validate
from surprise import Dataset
from surprise import Reader


In [2]:
reader = Reader(line_format='user item rating', sep=',', skip_lines=3, rating_scale=(1, 5))
data = Dataset.load_from_file(file_path='./ml-latest-small/ratings.csv', reader=reader)


In [3]:
neighbor_size=np.arange(5,100,10)


In [4]:

Knn_time = []
Knn_train_MAE=[]
Knn_test_MAE=[]
Knn_train_RMSE=[]
Knn_test_RMSE=[]

for k in neighbor_size:    
    algo=KNNBaseline(k=k)
    c_v=cross_validate(algo, data,cv=10, verbose=True,return_train_measures =True)
    Knn_time.append(np.mean(c_v['fit_time']))
    Knn_train_MAE.append(np.mean(c_v['train_mae']))
    Knn_test_MAE.append(np.mean(c_v['test_mae']))
    Knn_train_RMSE.append(np.mean(c_v['train_rmse']))
    Knn_test_RMSE.append(np.mean(c_v['test_rmse']))
    

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matr

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBaseline on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    0.8686  0.8780  0.8707  0.8798  0.8556  0.8640  0.8735  0.8609  0.8693  0.8772  0.8698  0.0074  
MAE (testset)     0.6683  0.6689  0.6628  0.6733  0.6538  0.6637  0.6645  0.6600  0.6655  0.6672  0.6648  0.0051  
RMSE (trainset)   0.6828  0.6815  0.6828  0.6817  0.6833  0.6829  0.6821  0.6835  0.6826  0.6821  0.6825  0.0006  
MAE (trainset)    0.5074  0.5073  0.5084  0.5077  0.5088  0.5

Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity

In [6]:
knn_result=[Knn_time,Knn_train_MAE,Knn_test_MAE,Knn_train_RMSE,Knn_test_RMSE]

In [25]:
learning_rate=np.arange(0.001,0.01,0.002)
regularization=np.arange(0.01,0.1,0.02)
n_factors=np.arange(50,200,50)
n_epochs=np.arange(10,100,20)

svd_result=[]
for lr in learning_rate:
    SVD_time = []
    SVD_train_MAE=[]
    SVD_test_MAE=[]
    SVD_train_RMSE=[]
    SVD_test_RMSE=[]
    
    for reg in regularization:
        algo=SVD(n_epochs =50,lr_all=lr,reg_all=reg)
        c_v=cross_validate(algo, data,cv=10, verbose=True,return_train_measures =True)
        SVD_time.append(np.mean(c_v['fit_time']))
        SVD_train_MAE.append(np.mean(c_v['train_mae']))
        SVD_test_MAE.append(np.mean(c_v['test_mae']))
        SVD_train_RMSE.append(np.mean(c_v['train_rmse']))
        SVD_test_RMSE.append(np.mean(c_v['test_rmse']))
    svd_result.append([SVD_time,SVD_train_MAE,SVD_test_MAE,SVD_train_RMSE,SVD_test_RMSE])


Evaluating RMSE, MAE of algorithm SVD on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
MAE (testset)     0.6680  0.6718  0.6604  0.6627  0.6696  0.6645  0.6662  0.6720  0.6608  0.6623  0.6658  0.0041  
RMSE (testset)    0.8695  0.8771  0.8588  0.8685  0.8669  0.8673  0.8699  0.8726  0.8653  0.8646  0.8681  0.0046  
Fit time          6.16    5.83    6.47    6.04    6.48    6.26    6.57    6.06    6.17    6.15    6.22    0.22    
Test time         0.24    0.08    0.12    0.08    0.07    0.13    0.08    0.13    0.10    0.10    0.11    0.05    


{'fit_time': (6.160496711730957,
  5.833416700363159,
  6.4746880531311035,
  6.0368568897247314,
  6.478678464889526,
  6.257303476333618,
  6.571396112442017,
  6.064783811569214,
  6.174485921859741,
  6.1465654373168945),
 'test_mae': array([0.66795522, 0.67180255, 0.66039578, 0.66267865, 0.66961794,
        0.66451582, 0.66615232, 0.67197841, 0.66081894, 0.66234616]),
 'test_rmse': array([0.86952007, 0.87711567, 0.85881971, 0.86845419, 0.86693562,
        0.86731204, 0.86993675, 0.87264264, 0.86534624, 0.86455772]),
 'test_time': (0.24337983131408691,
  0.07577991485595703,
  0.1157224178314209,
  0.07679486274719238,
  0.07081294059753418,
  0.12665390968322754,
  0.07878971099853516,
  0.126692533493042,
  0.10375189781188965,
  0.09671497344970703)}